#**Trabalho prático: Simulação e controle de um robô manipulador para paletização de caixas**

####Clona repositório do uaibot e instala biblioteca

In [ ]:
!git clone https://github.com/viniciusmgn/uaibot_vinicius.git
!pip install uaibot_vinicius/

fatal: destination path 'uaibot_vinicius' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./uaibot_vinicius
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for uaibot: filename=uaibot-0.0.1-py3-none-any.whl size=104850 sha256=b4804bbeac2cfe87470243eb6668236eeea301a4169f41fba64c7a71127b9c42
  Stored in directory: /tmp/pip-ephem-wheel-cache-tohvocrx/wheels/2a/63/33/ae58b9e82de4b97bb2ae16b73d17cb1cd7bd

####Importa bibliotecas necessárias para o desenvolvimento da simulação

In [ ]:
import uaibot as ub
import numpy as np
import math as m
import plotly as ply

####Cria cenário

*   Texturas
  * Alumínio
  * Plástico
  * Madeira
  * Concreto
*   Objetos
  * Esteira transportadora
  * Caixas
  * Palete
  * Base do robô



In [ ]:
from uaibot.graphics.meshmaterial import MeshMaterial

texture_conveyor = ub.Texture(
    url='https://raw.githubusercontent.com/IsabelaBraga96/TP_Manipuladores_Roboticos/master/Textura_aluminio.jpg',
    wrap_s='RepeatWrapping', wrap_t='RepeatWrapping', repeat=[1, 1])

texture_box = ub.Texture(
    url='https://raw.githubusercontent.com/IsabelaBraga96/TP_Manipuladores_Roboticos/master/Textura_plastico.jpg',
    wrap_s='RepeatWrapping', wrap_t='RepeatWrapping', repeat=[1, 1])

texture_wood = ub.Texture(
    url='https://raw.githubusercontent.com/IsabelaBraga96/TP_Manipuladores_Roboticos/master/Textura_madeira.jpg',
    wrap_s='RepeatWrapping', wrap_t='RepeatWrapping', repeat=[1, 1])

material_caixa = ub.MeshMaterial(texture_map=texture_box, roughness=1, metalness=0, opacity=1,color='#004d99',reflectivity=0, clearcoat=0, emissive='#3c3939')
material_correia = ub.MeshMaterial(texture_map=texture_conveyor, roughness=0.364, metalness=0.415, opacity=1,color='#4e4f50',reflectivity=1, clearcoat=0.16, emissive='#3c3939')
material_wood = ub.MeshMaterial(texture_map=texture_wood, roughness=0.364, metalness=0.7, opacity=1)

conveyor_1 = ub.Model3D(url = 'https://raw.githubusercontent.com/IsabelaBraga96/TP_Manipuladores_Roboticos/master/Correia_Expansiva.obj', scale= 0.005, mesh_material= material_correia)
box_1 = ub.Model3D(url = 'https://raw.githubusercontent.com/IsabelaBraga96/TP_Manipuladores_Roboticos/master/Caixa_Plastico.obj', scale= 0.0035, mesh_material= material_caixa)

conveyor = ub.RigidObject(name = 'conveyor', list_model_3d=[conveyor_1],htm = ub.Utils.trn([0.7,0.2,0]) @ ub.Utils.rotz(np.pi/2))

box = ub.RigidObject(name = 'box', list_model_3d=[box_1],htm = ub.Utils.trn([0.69,0.2,0.41]) @ ub.Utils.rotz(np.pi/2))
box1 = ub.RigidObject(name = 'box1', list_model_3d=[box_1],htm = ub.Utils.trn([0.69,0.2,0.41]) @ ub.Utils.rotz(np.pi/2))
box2 = ub.RigidObject(name = 'box2', list_model_3d=[box_1],htm = ub.Utils.trn([0.69,0.2,0.41]) @ ub.Utils.rotz(np.pi/2))
box3 = ub.RigidObject(name = 'box3', list_model_3d=[box_1],htm = ub.Utils.trn([0.69,0.2,0.41]) @ ub.Utils.rotz(np.pi/2))
box4 = ub.RigidObject(name = 'box4', list_model_3d=[box_1],htm = ub.Utils.trn([0.69,0.2,0.41]) @ ub.Utils.rotz(np.pi/2))
box5 = ub.RigidObject(name = 'box5', list_model_3d=[box_1],htm = ub.Utils.trn([0.69,0.2,0.41]) @ ub.Utils.rotz(np.pi/2))

base_pallet = ub.Box(name="base_pallet", htm = ub.Utils.trn([0,0.6,0.05]), width=0.6, depth=0.6, height=0.1, mesh_material=material_wood)

material_table = ub.MeshMaterial(roughness=0.35, metalness=1, opacity=1,color='#888B8D',reflectivity=1)
base_quadrado = ub.Box(name="base_quadrado", htm = ub.Utils.trn([0,0,0.05]), width=0.5, depth=0.5, height=0.10, mesh_material=material_table)
base_cilindro = ub.Cylinder(name='base_cilindro',htm = ub.Utils.trn([0,0,0.2]), radius=0.24, height=0.2, mesh_material=material_table)

#### Define os parâmetros da simulação e as funções de interface do robô

In [ ]:
dt = 0.01
t = 0
tmax = 70

def get_configuration(robot):
  return robot.q

def set_configuration_speed(robot, qdot_des):
  q_next = robot.q + qdot_des*dt
  robot.add_ani_frame(time = t+dt, q = q_next)

#### Cria robô, implementa lógica de controle e gera a simulação

In [ ]:
robot = ub.Robot.create_kuka_kr5(htm = ub.Utils.trn([0,0,base_quadrado.height+base_cilindro.height]), color = 'gray')
HOME = robot.fkm(robot.q0)
POS = box.htm

#Define os limites de junta e de velocidade de junta
limites_de_vel = [4.363,4.363,4.363,7.155,7.155,11.519]
limites_de_junta = [[2.967,-2.967],[0.785,-3.316],[2.879,-2.076],[3.316,-3.316],[2.094,-2.094],[6.248,-6.248]]

# Cria a função F:
def fun_F(r):
    A = [0.25, 0.25, 0.25, 0.25, 0.25, 0.25]
    w_tol = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
    F = np.matrix(np.zeros((6, 1)))
    for i in range(6):
        if abs(r[i, 0]) < w_tol[i]:
            F[i, 0] = -A[i] * (r[i, 0] / w_tol[i])
        elif r[i, 0] >= w_tol[i]:
            F[i, 0] = -A[i]
        else:
            F[i, 0] = A[i]
    return F

## Crias as posições para o Robô
h=0
htm_d = []
# Pega a primeira caixa
htm_d.append(POS @ ub.Utils.trn([0,0,0.3]) @ ub.Utils.rotx(m.pi)) #0
htm_d.append(base_pallet.htm @ ub.Utils.trn([0,-0.15,0.75]) @ ub.Utils.rotx(m.pi)) #1
htm_d.append(htm_d[len(htm_d)-1] @ ub.Utils.trn([0,0,0.53])) #3

# Pega a segunda caixa
htm_d.append(POS @ ub.Utils.trn([0,0,0.3]) @ ub.Utils.rotx(m.pi)) #4
htm_d.append(base_pallet.htm @ ub.Utils.trn([0,0.15,0.75]) @ ub.Utils.rotx(m.pi)) #5
htm_d.append(htm_d[len(htm_d)-1] @ ub.Utils.trn([0,0,0.53])) #6

# Pega a terceira caixa
htm_d.append(POS @ ub.Utils.trn([0,0,0.3]) @ ub.Utils.rotx(m.pi)) #7
htm_d.append(base_pallet.htm @ ub.Utils.trn([0,-0.15,0.75]) @ ub.Utils.rotx(m.pi)) #8
htm_d.append(htm_d[len(htm_d)-1] @ ub.Utils.trn([0,0,0.38])) #9

# Pega a quarta caixa
htm_d.append(POS @ ub.Utils.trn([0,0,0.3]) @ ub.Utils.rotx(m.pi)) #10
htm_d.append(base_pallet.htm @ ub.Utils.trn([0,0.15,0.75]) @ ub.Utils.rotx(m.pi)) #11
htm_d.append(htm_d[len(htm_d)-1] @ ub.Utils.trn([0,0,0.38])) #12

# Pega a quinta caixa
htm_d.append(POS @ ub.Utils.trn([0,0,0.3]) @ ub.Utils.rotx(m.pi)) #13
htm_d.append(base_pallet.htm @ ub.Utils.trn([0,-0.15,0.75]) @ ub.Utils.rotx(m.pi)) #14
htm_d.append(htm_d[len(htm_d)-1] @ ub.Utils.trn([0,0,0.23])) #15

# Pega a sexta caixa
htm_d.append(POS @ ub.Utils.trn([0,0,0.3]) @ ub.Utils.rotx(m.pi)) #16
htm_d.append(base_pallet.htm @ ub.Utils.trn([0,0.15,0.75]) @ ub.Utils.rotx(m.pi)) #17
htm_d.append(htm_d[len(htm_d)-1] @ ub.Utils.trn([0,0,0.23])) #18

htm_d.append(HOME) #19 - FIM

# Cria a simulação
d = ub.Frame(htm_d[4])
d1 = ub.Frame(htm_d[3])
#d2 = ub.Frame(htm_d[6])
#sim = ub.Simulation.create_sim_factory([conveyor, box, base_pallet])
#sim = ub.Simulation.create_sim_factory([robot, conveyor, base_quadrado,base_cilindro,box,base_pallet,d,d1,d2])
sim = ub.Simulation.create_sim_factory([robot, conveyor, base_quadrado,base_cilindro,box,base_pallet,d])
sim.add(box1)
sim.add(box2)
sim.add(box3)
sim.add(box4)
sim.add(box5)

# Captura o número de juntas do robô
n = np.shape(robot.q)[0]

# Cria uma matriz para o histórico de função de tarefa, da ação de controle
# e do tempo
hist_r = np.matrix(np.zeros((6,0)))
hist_u = np.matrix(np.zeros((n,0)))
hist_q = np.matrix(np.zeros((n,0)))
hist_t = []

#Colocaremos aqui nosso "main" do controlador, que ficará em um laço
#durante um tempo tmax
for i in range(round(tmax/dt)):
  #################################
  # Início da lógica de controle  #
  #################################

  # Mede a configuração dos sensores
  q = get_configuration(robot)

  # Calcula a cinemática direta e Jacobiana para o efetuador nessa
  # configuração
  Jg, fk = robot.jac_geo(q)
  #Faz a extração de x_e, y_e, z_e e s_e
  x_e = fk[0:3,0]
  y_e = fk[0:3,1]
  z_e = fk[0:3,2]
  s_e = fk[0:3,3]

  # Faz a extração dos elementos x_d, y_d, z_d e s_d
  x_d = htm_d[h][0:3,0]
  y_d = htm_d[h][0:3,1]
  z_d = htm_d[h][0:3,2]
  s_d = htm_d[h][0:3,3]

  # Monta o vetor de tarefa
  r = np.matrix(np.zeros((6,1)))
  r[0:3] = s_e - s_d
  r[3] = 1- x_d.T * x_e
  r[4] = 1- y_d.T * y_e
  r[5] = 1- z_d.T * z_e

  # Monta a Jacobiana de tarefa
  Jr = np.matrix(np.zeros((6,n)))

  Jr[0:3,:] = Jg[0:3,:]
  Jr[3,:] = x_d.T * ub.Utils.S(x_e) * Jg[3:6,:]
  Jr[4,:] = y_d.T * ub.Utils.S(y_e) * Jg[3:6,:]
  Jr[5,:] = z_d.T * ub.Utils.S(z_e) * Jg[3:6,:]

  # Calcula a ação de controle
  u = ub.Utils.dp_inv(Jr,0.001)*fun_F(r)

  # Limita a velocidade máxima de junta
  for j in range(n):
    if u[j] > limites_de_vel[j]:
      u[j] = limites_de_vel[j]
    elif u[j] < -limites_de_vel[j]:
      u[j] = -limites_de_vel[j]
    
  # Limita o movimento da junta
  for k in range(n):
    if (q[k] + u[k]*dt) > limites_de_junta[k][0]:
      u[k] = (limites_de_junta[k][0] - q[k])/dt
    elif (q[k] + u[k]*dt) < limites_de_junta[k][1]:
      u[k] = (limites_de_junta[k][0] - q[k])/dt

  # Guarda informações no histórico
  hist_r = np.block([hist_r, r])
  hist_u = np.block([hist_u, u])
  hist_q = np.block([hist_q, q])
  hist_t.append(t)

  # Manda a ação de controle para o robô
  set_configuration_speed(robot, u)

  #################################
  # Fim da lógica de controle     #
  #################################
  if(h<(len(htm_d)-1) and all(p <= 0.0001 for p in r)):
    h = h+1; #Nova posição
    # print(h)
    if (h==1):
      robot.attach_object(box)
    if(h==3):
      robot.detach_object(box)
    if(h==4):
      robot.attach_object(box1)
    if(h==6):
      robot.detach_object(box1)
    if(h==7):
      robot.attach_object(box2)
    if(h==9):
      robot.detach_object(box2)
    if(h==10):
      robot.attach_object(box3)
    if(h==12):
      robot.detach_object(box3)
    if(h==13):
      robot.attach_object(box4)
    if(h==15):
      robot.detach_object(box4)
    if(h==16):
      robot.attach_object(box5)
    if(h==18):
      robot.detach_object(box5)
  #O tempo sempre vai passar no final do ciclo
  t+=dt

# Roda a simulação
sim.run()

####Gera gráficos para análise

In [ ]:
# Plota gráficos das funções de tarefa e dos sinais de controle
ub.Utils.plot(hist_t, hist_r, "", "Tempo (s)", "Função de tarefa", "r")
ub.Utils.plot(hist_t, hist_u, "", "Tempo (s)", "Ação (rad/s ou m/s)", "u")

In [ ]:
# Plota gráficos das configurações das juntas e seus limites negativo e positivo
lim_pos_matrix = np.zeros((n,int(tmax/dt)))
lim_neg_matrix = np.zeros((n,int(tmax/dt)))

for i in range(n):
  lim_pos_matrix[i,:] = limites_de_junta[i][0]
  lim_neg_matrix[i,:] = limites_de_junta[i][1]

list_names = ["q_1","q_2","q_3","q_4","q_5","q_6"]
for i in range(n):
  fig = ply.express.line(width=800, height=400)
  fig.add_scatter(x=hist_t, y=np.matrix(hist_q[i,:]).tolist()[0], mode='lines', name=list_names[i])
  fig.add_scatter(x=hist_t, y=np.matrix(lim_pos_matrix[i,:]).tolist()[0], mode='markers',marker=dict(symbol='300',size=1,color='red'),name='limite positivo')
  fig.add_scatter(x=hist_t, y=np.matrix(lim_neg_matrix[i,:]).tolist()[0], mode='markers',marker=dict(symbol='300',size=1,color='orange'),name='limite negativo')
  fig.update_xaxes(title_text='Tempo (s)')
  fig.update_yaxes(title_text='Rotação da junta (rad)')
  fig.show()  